In [1]:
import pandas as pd
from config import *
from db_handler import *
from api_handler import *

### Setting up endpoint

In [43]:
headers = {
    'Access': 'application/json',
    'Authorization': FSQ_API_KEY,
}
params = {
    'ne': NE_FSQ,
    'sw': SW_FSQ,
    'categories': CATEGORY_FSQ,
    'limit': LIMIT_FSQ,
    'sort': SORT_FSQ
}
url = 'https://api.foursquare.com/v3/places/search?'

In [44]:
response = get_req(url, params=params, headers=headers)

In [45]:
fsq_ids_fsq = [result['fsq_id'] for result in response['results']]  # ids of the restaurants

In [46]:
params = {
    'fields': POI_RICH_DATA_FSQ
}
urls = [f"https://api.foursquare.com/v3/places/{id}" for id in fsq_ids_fsq]

In [47]:
json_responses = [get_req(url, params, FSQ_API_KEY) for url in urls]

In [49]:
len(json_responses)

15

### Preparing data to insert into DB

In [66]:
data_to_insert = [(result.get('fsq_id', None), 
                      result.get('name', None),
                       result.get('popularity', None),
                        result.get('rating', None),
                         result.get('website', None),
                          result.get('location', {}).get('formatted_address', None),
                           result.get('stats',{}).get('total_ratings', 0))
                               for result in json_responses]

In [51]:
rows_to_insert = ''
for new_query in data_to_insert:
    if new_query == data_to_insert[-1]:
        rows_to_insert += str(new_query)
    else:
        rows_to_insert += str(new_query) + ','

### Setting up SQLite table

### Creating Tables & Inserting Data

In [2]:
connection = create_connection("./data/coctail_spots_old_port.sqlite")

In [3]:
create__table = """
CREATE TABLE IF NOT EXISTS coctail_spots_old_port (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  fsq_id TEXT NOT NULL,
  name TEXT NOT NULL,
  popularity REAL,
  rating REAL,
  website TEXT,
  formatted_address TEXT,
  total_ratings INTEGER
);
"""

In [4]:
execute_query(connection, create__table)

In [58]:
create_spots = f"""
INSERT INTO
  coctail_spots_old_port (fsq_id, name, popularity, rating, website, formatted_address, total_ratings)
VALUES
    {rows_to_insert};
"""

In [59]:
execute_query(connection, create_spots)

### Requesting DB

In [5]:
select_coctail_spots = "SELECT * from coctail_spots_old_port"
coctail_spots = execute_read_query(connection, select_coctail_spots)

In [6]:
df = pd.DataFrame(coctail_spots, 
                  columns=['ID','fsq_id', 'name', 'popularity', 'rating',
                           'website', 'formatted_address', 'total_ratings'])

In [10]:
df[['name', 'rating', 'formatted_address']].sort_values('rating', ascending=False).head(10).to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>name</th>\n      <th>rating</th>\n      <th>formatted_address</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Bar le Mal Nécessaire</td>\n      <td>8.5</td>\n      <td>1106B Saint Laurent Blvd (coin René-Lévesque), Montréal QC H2Z 1J5</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Lab Quartier des Spectacles</td>\n      <td>8.5</td>\n      <td>279 Sainte-Catherine Rue E (coin Sanguinet), Montréal QC H2X 1L5</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Boho</td>\n      <td>8.3</td>\n      <td>436 Saint-Francois-Xavier Rue, Montréal QC H2Y 2T3</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Club Pelicano</td>\n      <td>7.9</td>\n      <td>1076 Rue de Bleury, Montréal QC H2Z 1N2</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Bord\'Elle</td>\n      <td>7.8</td>\n      <td>390 Saint-Jacques Rue, Montréal QC H2Y 1S1<